In [93]:
import h5py as hf
import pyretest as pr
import numpy as np
from scipy import interpolate
import pandas as pd

from mne_bids import BIDSPath, read_raw_bids
from mne_nirs.signal_enhancement import (enhance_negative_correlation,
                                         short_channel_regression)
from mne.preprocessing.nirs import optical_density, beer_lambert_law, scalp_coupling_index, temporal_derivative_distribution_repair
from mne.io import read_raw_snirf

In [94]:
n_subs = 2
ch_names = ["Resp.", "PPG", "Sp02", "HR", "GSR", "Temp."]

In [99]:

channel_lists = {}
# create a list for each channel
for channel in ch_names:
    # Use the channel name to create a variable name like 'resp_list' or 'ppg_list'
    list_name = f'{channel.lower()}_array'
    channel_lists[channel] = np.empty((0, 1))
    
times = []

for sub in range(1,n_subs+1):                
    subject_id = "%02d" % sub
    raw_path = BIDSPath(
        subject="%02d" % sub,
        task='wings',
        session='01',
        datatype="nirs",
        suffix='nirs',
        root=r"C:\Users\dalto\Downloads\project\sourcedata_lm",
        extension=".snirf"
    )

    raw_haemo = beer_lambert_law(optical_density(read_raw_snirf(raw_path)))
    raw_haemo = raw_haemo.filter(None, 0.4,
                                h_trans_bandwidth=0.1, l_trans_bandwidth=0.01,
                                verbose=False)
    raw_haemo.annotations.delete(raw_haemo.annotations.description == '15')
    raw_haemo = enhance_negative_correlation(raw_haemo)

    for n in (range(18, 24)):
        ch = ch_names[n-18]
        with hf.File(raw_path, 'r') as f:

            aux = np.array(f.get(f'nirs/aux{n}/dataTimeSeries'))
            aux_time = np.array(f.get(f'nirs/aux{n}/time'))

        aux_data_interp = interpolate.interp1d(aux_time, aux, axis=0, bounds_error=False, fill_value='extrapolate')
        aux_data_match = aux_data_interp(raw_haemo.times)

        channel_lists[ch] = np.append(channel_lists[ch], aux_data_match)

    times.append(raw_haemo.times)


Loading C:\Users\dalto\Downloads\project\sourcedata_lm\sub-01\ses-01\nirs\sub-01_ses-01_task-wings_nirs.snirf
Reading 0 ... 5829  =      0.000 ...  1145.981 secs...
Loading C:\Users\dalto\Downloads\project\sourcedata_lm\sub-02\ses-01\nirs\sub-02_ses-01_task-wings_nirs.snirf
Reading 0 ... 6250  =      0.000 ...  1228.750 secs...


In [101]:
df = pd.DataFrame()

# print(channel_lists['Resp.'][1].shape)
# times = times.flatten()

for i in range(6):
    df['Times'] = times[0]
    # channel_lists[ch_names[i]][i] = channel_lists[ch_names[i]][i].flatten()
    print(channel_lists[ch_names[i]][i].shape)
    df[ch_names[i]] = channel_lists[ch_names[i]][i]

()
()
()
()
()
()


In [102]:
display(df)

,Times,Resp.,PPG,Sp02,HR,GSR,Temp.
0,0.0000,58.297924,16.521999,99.0,54.0,0.01976,22.625
1,0.1966,58.297924,16.521999,99.0,54.0,0.01976,22.625
2,0.3932,58.297924,16.521999,99.0,54.0,0.01976,22.625
3,0.5898,58.297924,16.521999,99.0,54.0,0.01976,22.625
4,0.7864,58.297924,16.521999,99.0,54.0,0.01976,22.625
...,...,...,...,...,...,...,...
5825,1145.1950,58.297924,16.521999,99.0,54.0,0.01976,22.625
5826,1145.3916,58.297924,16.521999,99.0,54.0,0.01976,22.625
5827,1145.5882,58.297924,16.521999,99.0,54.0,0.01976,22.625
5828,1145.7848,58.297924,16.521999,99.0,54.0,0.01976,22.625
